In [ ]:
# Objective Evaluation
# Word Error Rate (WER):

# Transcribe the generated audio back to text using an automatic speech recognition (ASR) system.
# Compare the transcribed text with the original text to calculate the WER. The model with the lower WER is better.

In [7]:
from jiwer import wer

original_text = "The original text from the dataset."
transcribed_text_model1 = "The original text from the database."
transcribed_text_model2 = "The transcribed text from model 2."

wer_model1 = wer(original_text, transcribed_text_model1)
wer_model2 = wer(original_text, transcribed_text_model2)

print(f"WER Model 1: {wer_model1}")
print(f"WER Model 2: {wer_model2}")


WER Model 1: 0.16666666666666666
WER Model 2: 0.5


In [ ]:
# Perceptual Evaluation of Speech Quality (PESQ):

# Use PESQ to measure the quality of the speech signals. PESQ compares the generated audio to a reference audio signal and gives a quality score.
# Libraries like pypesq can be used for this purpose.

In [10]:
from pypesq import pesq
from scipy.io import wavfile

rate, ref_audio = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac.wav')
rate, gen_audio_model1 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac.wav')
rate, gen_audio_model2 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.flac.wav')

pesq_score_model1 = pesq(rate, ref_audio, gen_audio_model1, 'wb')
pesq_score_model2 = pesq(rate, ref_audio, gen_audio_model2, 'wb')

print(f"PESQ Score Model 1: {pesq_score_model1}")
print(f"PESQ Score Model 2: {pesq_score_model2}")


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()